# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [15]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_02_full_spectral.pkl".format(OUTPUT_PREFIX))


# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral_copy.pkl")
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")

In [16]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [17]:
theta_band = (4, 12)
gamma_band = (30, 50)
filter_band = (0, 56)

In [18]:
FONTSIZE=20

In [19]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Dropping unused columns

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
list(LFP_SPECTRAL_DF.columns)

['cohort',
 'session_dir',
 'tone_frames',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'notes',
 'session_path',
 'recording',
 'current_subject',
 'subject',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'video_timestamps',
 'tone_timestamps',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'lfp_timestamps',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'mPFC_lfp_RMS_filtered',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_power_all_frequencies

In [22]:
dropping_columns = [col for col in LFP_SPECTRAL_DF if ("MAD" in col or "modified_zscore" in col or "RMS" in col or "mask" in col or "trace_filtered" in col) and "power" not in col]




In [23]:
dropping_columns

['vHPC_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'mPFC_lfp_RMS_filtered']

In [24]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

## Filtering frequencies

In [25]:
list(LFP_SPECTRAL_DF.columns)

['cohort',
 'session_dir',
 'tone_frames',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'notes',
 'session_path',
 'recording',
 'current_subject',
 'subject',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'video_timestamps',
 'tone_timestamps',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'lfp_timestamps',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_

In [26]:
all_frequencies_columns = [col for col in LFP_SPECTRAL_DF if "all_frequencies_all_windows" in col] 

In [27]:
for col in all_frequencies_columns:
    if "power" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["power_calculation_frequencies"] >= filter_band[0]) & (x["power_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["power_calculation_frequencies"] = LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [28]:
for col in all_frequencies_columns:
    if "coherence" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["coherence_calculation_frequencies"] >= filter_band[0]) & (x["coherence_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["coherence_calculation_frequencies"] = LFP_SPECTRAL_DF["coherence_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [29]:
for col in all_frequencies_columns:
    if "granger" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["granger_calculation_frequencies"] >= filter_band[0]) & (x["granger_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["granger_calculation_frequencies"] = LFP_SPECTRAL_DF["granger_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [30]:
for col in all_frequencies_columns:
    print(LFP_SPECTRAL_DF[col].iloc[0].shape)

(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)
(6358, 56)


In [31]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].iloc[0].shape

(56,)

In [32]:
LFP_SPECTRAL_DF["coherence_calculation_frequencies"].iloc[0].shape

(56,)

In [33]:
LFP_SPECTRAL_DF["granger_calculation_frequencies"].iloc[0].shape

(56,)

## Filtering by theta and gamma band

- Filtering theta/gamma for coherence

In [34]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [35]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows']

In [36]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


In [37]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,LH_mPFC_coherence_theta,LH_mPFC_coherence_gamma,LH_vHPC_coherence_theta,LH_vHPC_coherence_gamma,MD_mPFC_coherence_theta,MD_mPFC_coherence_gamma,MD_vHPC_coherence_theta,MD_vHPC_coherence_gamma,mPFC_vHPC_coherence_theta,mPFC_vHPC_coherence_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, nan, 0.66578615, 0.71455723, 0.70779204,...","[nan, nan, 0.42835087, 0.5590993, 0.43337494, ...","[nan, nan, 0.4717538, 0.4339342, 0.5687888, 0....","[nan, nan, 0.45298973, 0.22820581, 0.3585577, ...","[0.53791547, 0.7005945, 0.58507025, 0.5095136,...","[0.43646446, 0.49723727, 0.4331182, 0.53396994...","[nan, 0.6052197, 0.4935742, 0.6154693, nan, na...","[nan, 0.28264648, 0.47072393, 0.35121575, nan,...","[nan, 0.69276094, 0.67357486, 0.33313167, 0.53...","[nan, 0.24414322, 0.34572023, 0.3490095, 0.245..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, nan, 0.66578615, 0.71455723, 0.70779204,...","[nan, nan, 0.42835087, 0.5590993, 0.43337494, ...","[nan, nan, 0.4717538, 0.4339342, 0.5687888, 0....","[nan, nan, 0.45298973, 0.22820581, 0.3585577, ...","[0.53791547, 0.7005945, 0.58507025, 0.5095136,...","[0.43646446, 0.49723727, 0.4331182, 0.53396994...","[nan, 0.6052197, 0.4935742, 0.6154693, nan, na...","[nan, 0.28264648, 0.47072393, 0.35121575, nan,...","[nan, 0.69276094, 0.67357486, 0.33313167, 0.53...","[nan, 0.24414322, 0.34572023, 0.3490095, 0.245..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.8827801, 0.8987765, 0.9303894, nan, nan, 0....","[0.54208976, 0.7324878, 0.70628226, nan, nan, ...","[0.7073132, 0.8735628, 0.86064696, nan, nan, 0...","[0.6243902, 0.6751488, 0.72943676, nan, nan, 0...","[0.8822864, 0.93042713, 0.94797623, nan, nan, ...","[0.59913385, 0.74510723, 0.76855767, nan, nan,...","[0.6591146, 0.8446648, 0.8442039, nan, nan, 0....","[0.49413526, 0.67826587, 0.7268031, nan, nan, ...","[0.7519526, 0.7967754, 0.85469705, nan, nan, 0...","[0.4529712, 0.59342456, 0.6972208, nan, nan, 0..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.8827801, 0.8987765, 0.9303894, nan, nan, 0....","[0.54208976, 0.7324878, 0.70628226, nan, nan, ...","[0.7073132, 0.8735628, 0.86064696, nan, nan, 0...","[0.6243902, 0.6751488, 0.72943676, nan, nan, 0...","[0.8822864, 0.93042713, 0.94797623, nan, nan, ...","[0.59913385, 0.74510723, 0.76855767, nan, nan,...","[0.6591146, 0.8446648, 0.8442039, nan, nan, 0....","[0.49413526, 0.67826587, 0.7268031, nan, nan, ...","[0.7519526, 0.7967754, 0.85469705, nan, nan, 0...","[0.4529712, 0.59342456, 0.6972208, nan, nan, 0..."
4,rce_pilot_3,20240320_171038_alone_comp

In [38]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
video_name
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windows
mPFC_power_all_frequencies_all_windows
power_timestamps
power_calculation_frequencies
BLA_LH_coherence_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BL

- Filtering theta/gamma for granger

In [39]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [40]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows']

In [41]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA-to-LH
LH-to-BLA
BLA-to-MD
MD-to-BLA
BLA-to-mPFC
mPFC-to-BLA
BLA-to-vHPC
vHPC-to-BLA
LH-to-MD
MD-to-LH
LH-to-mPFC
mPFC-to-LH
LH-to-vHPC
vHPC-to-LH
MD-to-mPFC
mPFC-to-MD
MD-to-vHPC
vHPC-to-MD
mPFC-to-vHPC
vHPC-to-mPFC


In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,mPFC-to-MD_granger_theta,mPFC-to-MD_granger_gamma,MD-to-vHPC_granger_theta,MD-to-vHPC_granger_gamma,vHPC-to-MD_granger_theta,vHPC-to-MD_granger_gamma,mPFC-to-vHPC_granger_theta,mPFC-to-vHPC_granger_gamma,vHPC-to-mPFC_granger_theta,vHPC-to-mPFC_granger_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[1.025482, 0.41221225, 0.32672292, 0.1874097, ...","[0.60007405, 0.2822732, 0.29430756, 0.40580773...","[nan, 0.491483, 0.44139874, 0.53886676, nan, n...","[nan, 0.2496434, 0.21510589, 0.42774624, nan, ...","[nan, 1.2430829, 1.0742137, 0.7353348, nan, na...","[nan, 0.4479247, 0.54432523, 0.28272176, nan, ...","[nan, 0.9007437, 0.685132, 0.19129051, 0.80936...","[nan, 0.20879993, 0.26942345, 0.47420612, 0.16...","[nan, 0.4939099, 0.8372965, 0.64537555, 0.5764...","[nan, 0.50144595, 0.30817866, 0.14948007, 0.18..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[1.025482, 0.41221225, 0.32672292, 0.1874097, ...","[0.60007405, 0.2822732, 0.29430756, 0.40580773...","[nan, 0.491483, 0.44139874, 0.53886676, nan, n...","[nan, 0.2496434, 0.21510589, 0.42774624, nan, ...","[nan, 1.2430829, 1.0742137, 0.7353348, nan, na...","[nan, 0.4479247, 0.54432523, 0.28272176, nan, ...","[nan, 0.9007437, 0.685132, 0.19129051, 0.80936...","[nan, 0.20879993, 0.26942345, 0.47420612, 0.16...","[nan, 0.4939099, 0.8372965, 0.64537555, 0.5764...","[nan, 0.50144595, 0.30817866, 0.14948007, 0.18..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.09776501, 0.40236086, 0.28928506, nan, nan,...","[0.27274314, 0.2917483, 0.20819838, nan, nan, ...","[0.35391855, 0.5397556, 0.4918575, nan, nan, 0...","[0.3308991, 0.24682036, 0.23084931, nan, nan, ...","[0.1520014, 0.108759224, 0.18117219, nan, nan,...","[0.1207855, 0.28944397, 0.20289902, nan, nan, ...","[0.6886265, 1.288887, 0.5724063, nan, nan, 0.5...","[0.26781583, 0.17785063, 0.42926326, nan, nan,...","[0.23952945, 0.1226715, 0.2661613, nan, nan, 0...","[0.2798054, 0.47546524, 0.33869317, nan, nan, ..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.09776501, 0.40236086, 0.28928506, nan, nan,...","[0.27274314, 0.2917483, 0.20819838, nan, nan, ...","[0.35391855, 0.5397556, 0.4918575, nan, nan, 0...","[0.3308991, 0.24682036, 0.23084931, nan, nan, ...","[0.1520014, 0.108759224, 0.18117219, nan, nan,...","[0.1207855, 0.28944397, 0.20289902, nan, nan, ...","[0.6886265, 1.288887, 0.5724063, nan, nan, 0.5...","[0.26781583, 0.17785063, 0.42926326, nan, nan,...","[0.23952945, 0.1226715, 0.2661613, nan, nan, 0...","[0.2798054, 0.47546524, 0.33869317, nan, nan, ..."
4,rce_pilot_3,20240320_171

- Filtering theta/gamma for power

In [43]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [44]:
power_columns

['vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'mPFC_power_all_frequencies_all_windows']

In [45]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

array([False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [46]:
for col in power_columns:
    # brain_region_name = col.split("_")[0]
    brain_region_name = col.split("power")[0]
    brain_region_name = brain_region_name.strip("_")
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

vHPC
BLA
LH
MD
mPFC


In [47]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84

In [48]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [49]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
video_name
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windows
mPFC_power_all_frequencies_all_windows
power_timestamps
power_calculation_frequencies
BLA_LH_coherence_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BL

In [50]:
raise ValueError()

ValueError: 